<a href="https://colab.research.google.com/github/allenai/vampire/blob/colab/colab/VAMPIRE_AGNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### VAMPIRE Example: AG News Corpus

In this notebook, we run through the example in the README. Since VAMPIRE is a low resource method, it can be run on the CPU or Colab GPU. Before starting, open this notebook in a Colab environment, and connect to a GPU instance.

Clone the repository and cd into working directory:

In [0]:
!git clone https://github.com/allenai/vampire


In [0]:
%cd vampire

Install requirements:

In [0]:
!pip install -r requirements.txt

In [0]:
!python -m spacy download en

All tests should pass:

In [0]:
!SEED=42 python -m pytest -v --color=yes vampire

Download preprocessed AG News corpus, ready to run with VAMPIRE:

In [0]:
!sh scripts/download_ag.sh
!curl -Lo ag.tar https://s3-us-west-2.amazonaws.com/allennlp/datasets/ag-news/vampire_preprocessed_example.tar
!tar -xvf ag.tar -C examples/
!rm ag.tar

Run VAMPIRE:

In [0]:
!DATA_DIR="$(pwd)/examples/ag" VOCAB_SIZE=30000 LAZY=1 python -m scripts.train \
            --config training_config/vampire.jsonnet \
            --serialization-dir model_logs/vampire \
            --environment VAMPIRE \
            --device 0 -o

After VAMPIRE has trained, we can run a downstream classifier on the AG News corpus with just 200 examples. We report an average of 83.9% accuracy in the paper over five seeds under this setting:

In [0]:
!DATA_DIR="$(pwd)/examples/ag" VAMPIRE_DIR="$(pwd)/model_logs/vampire" VAMPIRE_DIM=81 THROTTLE=200 EVALUATE_ON_TEST=0 python -m scripts.train \
            --config training_config/classifier.jsonnet \
            --serialization-dir model_logs/clf \
            --environment CLASSIFIER \
            --device 0